In [4]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import joblib

# Load the trained SVM model and PCA model
svm = SVC(kernel='linear')
svm = joblib.load(r'D:\MTECH\PROJECT\CV\ORIGINAL\FINAL\computer-vision-hands-on\Project\Models\svm_model.joblib')
pca = PCA(n_components=100)
pca = joblib.load(r'D:\MTECH\PROJECT\CV\ORIGINAL\FINAL\computer-vision-hands-on\Project\Models\pca_model.joblib')

# Set up the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    face_cascade = cv2.CascadeClassifier(r'D:\MTECH\PROJECT\CV\ORIGINAL\FINAL\computer-vision-hands-on\Project\Haarcascade\haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # Loop over each face
    for (x, y, w, h) in faces:
        # Extract the face ROI
        roi_gray = gray[y:y+h, x:x+w]

        # Resize the ROI to 48x48 pixels
        roi_gray = cv2.resize(roi_gray, (48, 48))

        # Extract LBP features from the ROI
        lbp = local_binary_pattern(roi_gray, 8, 1) # LBP parameters: (8, 1) for 8 neighbors and radius 1 for a total of 8*1=8 pixels
        hist, _ = np.histogram(lbp, bins=np.arange(0, 257), density=True) # Normalize the histogram values before feeding it to the SVM model
        lbp_features = np.array(hist).reshape(1, -1) # Convert the histogram to a feature vector using numpy array's reshape() method

        # Apply PCA to reduce the dimensionality of the LBP features
        lbp_pca = pca.transform(lbp_features)

        # Predict the emotion using the SVM model
        emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        predicted_emotion = svm.predict(lbp_pca)[0] # The predicted emotion  is the first element in the returned array

        # Draw the bounding box and label for the detected face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2) # Bounding box around the face ROI
        cv2.putText(frame, emotion_labels[predicted_emotion], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2) # Emotion label near the bounding box

    # Display the resulting frame
    cv2.imshow('LBP FACE EMOTION RECOGNITION', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

: 